In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('/home/dataset/Gowalla/loc-gowalla_totalCheckins.txt',
                delimiter = '\t',
                names = ['id', 'time', 'lat', 'lon', 'loc_id'],
                parse_dates = ['time'],
                index_col = ['time'])

df['hour'] = df.index.hour
df['day'] = df.index.day
df['new_time'] = df.index.floor('60min')
df['weekday'] = df.index.weekday

df = df.sort_values(by = ['id', 'time'])

df.head(10)

,id,lat,lon,loc_id,hour,day,new_time,weekday
time,,,,,,,,
2010-05-22 02:49:04,0,30.248924,-97.749626,608105,2,22,2010-05-22 02:00:00,5
2010-05-22 17:50:55,0,39.297443,-94.716053,8977,17,22,2010-05-22 17:00:00,5
2010-05-22 19:13:12,0,38.985246,-94.605919,18574,19,22,2010-05-22 19:00:00,5
2010-05-23 16:50:50,0,39.093533,-94.593174,17269,16,23,2010-05-23 16:00:00,6
2010-05-23 17:51:57,0,39.093258,-94.593871,1161876,17,23,2010-05-23 17:00:00,6
2010-05-23 22:40:45,0,39.000447,-94.577709,1163401,22,23,2010-05-23 22:00:00,6
2010-05-23 23:13:17,0,39.001931,-94.579466,668208,23,23,2010-05-23 23:00:00,6
2010-05-26 14:16:56,0,30.267910,-97.749312,21714,14,26,2010-05-26 14:00:00,2
2010-05-27 22:39:52,0,30.269103,-97.749395,420315,22,27,2010-05-27 22:00:00,3


In [29]:
import gensim.models as g
import sys
import os

class TDocsbyId(object):
    def __init__(self, person_limit = sys.maxsize):
        self.person_limit = person_limit
        
    def __iter__(self):
        for idx, (tags, group) in enumerate(df.groupby(['id'])):
            tags = [str(tags)]
            group['word'] = group['loc_id'].astype(str) + "_" + group['weekday'].astype(str) + "_" + group['hour'].astype(str)
            doc =g.doc2vec.TaggedDocument(words = group['word'].values, tags = tags)
            yield doc
            
            if idx >= self.person_limit:
                break
            
from gensim.models.callbacks import CallbackAny2Vec

class EpochSaver(CallbackAny2Vec):
    def __init__(self, down_sample = 50):
        self.epoch = 0
        self.down_sample = down_sample
    
    def on_epoch_begin(self, model):
        if self.epoch % self.down_sample == 0:
            print("Epoch #{} start".format(self.epoch))
        
    def on_epoch_end(self, model):
        if self.epoch % self.down_sample == 0:
            print("Epoch #{} end".format(self.epoch))
            
        os.makedirs('/opt/models', mode = 0o777, exist_ok = True)
        model.save("/opt/models/{}_epoch".format(self.epoch))
        self.epoch += 1

def doc2vec_train_byid():
    train_doc = TDocsbyId(10)
    model = g.doc2vec.Doc2Vec(dm = 1,
                             vector_size = 300,
                             min_count = 2,
                             epochs = 300,
                             windows =10,
                             alpha =0.025,
                             workers = 4,
                             samples = 1e-3)
    print("build vocab...")
    %time model.build_vocab(train_doc)
    print("start_train...")
    %time model.train(train_doc,total_examples = model.corpus_count,epochs = model.epochs,callbacks = [EpochSaver()])
    return model

In [22]:
# 开始训练
model = doc2vec_train_byid()

build vocab...


/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


CPU times: user 978 ms, sys: 136 ms, total: 1.11 s
Wall time: 1.11 s
start_train...
Epoch #0 start
Epoch #0 end


/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

...

Epoch #250 end


/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

CPU times: user 4min 57s, sys: 38.7 s, total: 5min 36s
Wall time: 5min 35s


In [35]:
def find_most_sim(model, words):
    try:
        return model.wv.most_similar(x, topn = 10)
    except:
        pass


def get_embeded_vecs(model):
    dfx = df.head(10000)

    dfx['word'] = dfx['loc_id'].astype(str) + "_" + dfx['weekday'].astype(str) + "_" + dfx['hour'].astype(str)
    dfx = dfx.groupby(['id', 'day', 'weekday']).agg({
        'word' : lambda x : list(x)
    }).reset_index()
    
    
    dfx['new_word'] = dfx['word'].apply(lambda x : model.infer_vector(x, alpha = 0.025, steps = 300))
    dfx['most_sim'] = dfx['word'].apply(lambda x : find_most_sim(model, x))
#    model.most_similar
    
    model.wv.most_similar
         
    return dfx

dfx = get_embeded_vecs(model)
dfx

/root/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,id,day,weekday,word,new_word,most_sim
0,0,1,1,[480992_1_14],"[0.15523969, 0.08760309, -0.20602642, 0.133464...",None
1,0,1,2,"[18417_2_1, 1067829_2_22]","[-0.0011176545, 0.00134668, -0.0007670643, -0....",None
2,0,1,4,[21714_4_14],"[0.15020138, 0.0928469, -0.21182622, 0.1447666...",None
3,0,1,6,[9508_6_2],"[-0.0013760627, -0.00015008997, 0.0015761802, ...",None
4,0,2,2,"[15326_2_0, 420315_2_15]","[0.18607822, 0.10463709, -0.23739988, 0.143647...",None
5,0,2,4,[18328_4_17],"[0.0014800066, 0.00057036686, -0.001519362, 0....",None
6,0,2,5,"[4256132_5_1, 1151119_5_22]","[0.14942354, 0.087355874, -0.20191994, 0.13268...",None
7,0,3,1,"[87899_1_1, 21714_1_19]","[0.13966686, 0.08869548, -0.19169647, 0.134313...",None
8,0,3,3,[155957_3_22],"[0.0008537915, 0.0002604287, -0.0007752067, -0...",None
9,0,3,4,[9313_4_0],"[-0.000922054, -0.0015060162, -0.00024887556, ...",None
